In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [ ]:
import pathlib
data_dir_path = "/kaggle/input/dataset/normalize_data"
data_dir = pathlib.Path(data_dir_path)
print(data_dir)

In [ ]:
meat_img = list(data_dir.glob('meat/*'))
print(meat_img[0])
PIL.Image.open(str(meat_img[0]))

In [ ]:
img_height,img_width=224,224
batch_size=64

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
class_numbers = label_encoder.fit_transform(class_names)
print(class_numbers)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(6):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_numbers[labels[i]])
    plt.axis("off")

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
def lr_update(epoch, lr):
    """
    For the first 5 epochs the learning rate will be 0.005.
    From epoch 6 and on, the learning rate will be reduced 1% per epoch
    """
    if epoch <= 5:
        return 0.005
    else:
        return lr * 0.99
    
lr_scheduler = LearningRateScheduler(lr_update)

from tensorflow.keras.callbacks import Callback

#creates a custom callback class
class CustomCallback(Callback):
    """
    This callback will stop the model from training once the model reaches 95% accuracy on the training data
    """
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') > 0.95:
            print('Accuracy above 95% -- Stopping Training')
            self.model.stop_training = True #stops model training

my_callback = CustomCallback()

In [99]:
from tensorflow.keras import regularizers
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',
                    classes=3,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(BatchNormalization(axis= -1 , momentum= 0.99 , epsilon= 0.001))
resnet_model.add(Flatten())
resnet_model.add(Dense(
        1024, 
        kernel_regularizer=regularizers.l2(0.016),
        activity_regularizer=regularizers.l1(0.006),
        bias_regularizer=regularizers.l1(0.006),
        activation='relu',
    ))
resnet_model.add(BatchNormalization(axis= -1 , momentum= 0.99 , epsilon= 0.001))
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(256, activation='relu'))
resnet_model.add(Dense(3, activation='softmax'))

resnet_model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ ?                      │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

In [101]:
resnet_model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [102]:
epochs=20
history = resnet_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
    batch_size=64,
    callbacks=[my_callback, lr_scheduler]
)

Epoch 1/20
728/728 ━━━━━━━━━━━━━━━━━━━━ 70s 78ms/step - accuracy: 0.6681 - loss: 22.2792 - val_accuracy: 0.6500 - val_loss: 9.8175 - learning_rate: 0.0050
Epoch 2/20
728/728 ━━━━━━━━━━━━━━━━━━━━ 46s 63ms/step - accuracy: 0.7093 - loss: 9.0334 - val_accuracy: 0.7074 - val_loss: 4.8929 - learning_rate: 0.0050
Epoch 3/20
728/728 ━━━━━━━━━━━━━━━━━━━━ 46s 63ms/step - accuracy: 0.8122 - loss: 3.2712 - val_accuracy: 0.8241 - val_loss: 1.8099 - learning_rate: 0.0050
Epoch 4/20
728/728 ━━━━━━━━━━━━━━━━━━━━ 46s 63ms/step - accuracy: 0.8998 - loss: 1.2097 - val_accuracy: 0.7785 - val_loss: 1.1959 - learning_rate: 0.0050
Epoch 5/20
728/728 ━━━━━━━━━━━━━━━━━━━━ 46s 63ms/step - accuracy: 0.9159 - loss: 0.7239 - val_accuracy: 0.8940 - val_loss: 0.6413 - learning_rate: 0.0050
Epoch 6/20
728/728 ━━━━━━━━━━━━━━━━━━━━ 46s 63ms/step - accuracy: 0.9228 - loss: 0.6042 - val_accuracy: 0.7796 - val_loss: 1.6556 - learning_rate: 0.0050
Epoch 7/20
728/728 ━━━━━━━━━━━━━━━━━━━━ 46s 63ms/step - accuracy: 0.9261 - 

In [ ]:
fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()
     

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.grid()
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
import cv2

normal = list(data_dir.glob('meat/*'))
print(normal[10])

image=cv2.imread(str(normal[10]))
image_resized= cv2.resize(image, (img_height,img_width))
image=np.expand_dims(image_resized,axis=0)
print(image.shape)
pred=resnet_model.predict(image)
print(pred)

In [ ]:
output_class=class_names[np.argmax(pred)]
print("The predicted class is", output_class)

In [ ]:
resnet_model.save('new_model.keras')